<a href="https://colab.research.google.com/github/Rajeevkulkarni1111/fall_detection/blob/main/fall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# MediaPipe setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Fall detection logic
def detect_fall(landmarks):
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    if left_hip.visibility < 0.5 or right_hip.visibility < 0.5:
        return False
    avg_hip_y = (left_hip.y + right_hip.y) / 2
    return avg_hip_y > 0.75  # Threshold for "fall"

# Test on video dataset file (MP4)
def test_on_video(video_path):
    cap = cv2.VideoCapture(video_path)  # <-- Here you give your video file path!
    fall_detected = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            if detect_fall(results.pose_landmarks.landmark):
                fall_detected = True
                cv2.putText(frame, 'FALL DETECTED!', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 4)
            else:
                cv2.putText(frame, 'NO FALL', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 4)
        else:
            cv2.putText(frame, 'NO PERSON DETECTED', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 4)

        cv2.imshow("Fall Detection on Video", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    if fall_detected:
        print("🚨 FALL DETECTED in video!")
    else:
        print("✅ No fall detected in video.")

# Dummy training function
def train_model():
    print("🧠 Training model on dummy video dataset... (simulated)")
    for i in range(3):
        print(f"📈 Training epoch {i+1}/3...")
        time.sleep(1)
    print("✅ Training complete! Model ready.")

# Webcam test with 10-sec timer
def webcam_test():
    print("🎥 Starting 10-second webcam session for fall detection...")
    cap = cv2.VideoCapture(0)
    start_time = time.time()
    fall_detected = False

    while time.time() - start_time < 10:
        ret, frame = cap.read()
        if not ret:
            print("❌ Failed to grab frame.")
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            if detect_fall(results.pose_landmarks.landmark):
                fall_detected = True
                cv2.putText(frame, 'FALL DETECTED!', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 4)
            else:
                cv2.putText(frame, 'NO FALL', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 4)
        else:
            cv2.putText(frame, 'NO PERSON DETECTED', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 4)

        cv2.imshow("Fall Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    if fall_detected:
        print("🚨 FALL DETECTED during webcam test!")
    else:
        print("✅ No fall detected in 10 seconds.")

# Main menu
def main():
    print("❤️ Fall Detection Menu:")
    print("1️⃣ Train")
    print("2️⃣ Test (Webcam 10s)")
    print("3️⃣ Test on video file")
    choice = input("Enter your choice (1, 2 or 3): ").strip()
    if choice == "1":
        train_model()
    elif choice == "2":
        webcam_test()
    elif choice == "3":
        video_path = input("📁 Enter the path to your video file (MP4): ").strip()
        test_on_video(video_path)
    else:
        print("❌ Invalid choice, cheeky! Try again with 1, 2 or 3!")

# Run
if __name__ == "__main__":
    main()

❤️ Fall Detection Menu:
1️⃣ Train
2️⃣ Test (Webcam 10s)
3️⃣ Test on video file
Enter your choice (1, 2 or 3): 3
📁 Enter the path to your video file (MP4): /content/walk1.mp4
✅ No fall detected in video.


In [ ]:
# Cell 1: Install dependencies
#!pip install mediapipe opencv-python-headless

# Cell 2: Import required libraries
import cv2
import time
import numpy as np
import mediapipe as mp
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from io import BytesIO
from PIL import Image

# Cell 3: Helper function to show webcam in Colab
def capture_webcam_frame():
    display(Javascript('''
        async function captureFrame() {
          const div = document.createElement('div');
          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});
          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          // Wait for 10 seconds or until 'q' is pressed
          await new Promise((resolve) => {
            const stopButton = document.createElement('button');
            stopButton.textContent = '📷 Stop';
            div.appendChild(stopButton);
            stopButton.onclick = () => {
              stream.getVideoTracks()[0].stop();
              video.remove();
              stopButton.remove();
              div.remove();
              resolve();
            };
            setTimeout(() => {
              stream.getVideoTracks()[0].stop();
              video.remove();
              stopButton.remove();
              div.remove();
              resolve();
            }, 10000);  // 10 seconds
          });

          // Draw image to canvas
          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          const data = canvas.toDataURL('image/jpeg');
          return data;
        }
        google.colab.output.setIframeHeight(0, true);
        captureFrame();
    '''))
    data = eval_js('captureFrame()')
    header, encoded = data.split(",", 1)
    binary = b64decode(encoded)
    image = Image.open(BytesIO(binary))
    return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

# Cell 4: Pose detection and fall logic
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

def detect_fall(landmarks):
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    if left_hip.visibility < 0.5 or right_hip.visibility < 0.5:
        return False
    avg_hip_y = (left_hip.y + right_hip.y) / 2
    return avg_hip_y > 0.75

def train_model():
    print("🧠 Training model (simulated)...")
    for i in range(3):
        print(f"Epoch {i+1}/3")
        time.sleep(1)
    print("✅ Model training complete.")

def webcam_test_colab():
    print("🎥 Opening webcam for 10 seconds or until you click 'Stop'...")
    img = capture_webcam_frame()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    if results.pose_landmarks:
        if detect_fall(results.pose_landmarks.landmark):
            print("🚨 FALL DETECTED!")
        else:
            print("✅ No fall detected.")
    else:
        print("🧍 No person detected.")

def test_on_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fall_detected = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)
        if results.pose_landmarks:
            if detect_fall(results.pose_landmarks.landmark):
                fall_detected = True
                print("🚨 Fall detected in video!")
        cv2.imshow('Video Fall Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    if not fall_detected:
        print("✅ No fall detected in video.")

# Cell 5: Menu
def main():
    print(" Fall Detection Menu:")
    print("1️⃣ Train")
    print("2️⃣ Test with Webcam")
    print("3️⃣ Test on Video File")
    choice = input("Enter your choice (1/2/3): ").strip()

    if choice == "1":
        train_model()
    elif choice == "2":
        webcam_test_colab()
    elif choice == "3":
        video_path = input("Enter path to video file (.mp4): ").strip()
        test_on_video(video_path)
    else:
        print("❌ Invalid choice. Please enter 1, 2 or 3.")

# Run the menu
main()


❤️ Fall Detection Menu:
1️⃣ Train
2️⃣ Test with Webcam
3️⃣ Test on Video File
Enter your choice (1/2/3): 2
🎥 Opening webcam for 10 seconds or until you click 'Stop'...


<IPython.core.display.Javascript object>

🧍 No person detected.


In [2]:
# Cell 1: Install dependencies
# !pip install mediapipe opencv-python-headless

# Cell 2: Imports
import cv2
import time
import numpy as np
import mediapipe as mp
from IPython.display import display, Javascript
from google.colab.output import eval_js
from IPython.display import Image
import io
import base64
from google.colab.patches import cv2_imshow

# Cell 3: JS-based snapshot function
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const capture = document.createElement('button');
          capture.textContent = '📸 Take Photo';
          div.appendChild(capture);

          const video = document.createElement('video');
          video.style.display = 'block';
          div.appendChild(video);

          document.body.appendChild(div);

          const stream = await navigator.mediaDevices.getUserMedia({ video: true });

          video.srcObject = stream;
          await video.play();

          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          await new Promise((resolve) => capture.onclick = resolve);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getTracks().forEach(track => track.stop());
          div.remove();

          const dataUrl = canvas.toDataURL('image/jpeg', quality);
          return dataUrl;
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = io.Bytes.io(base64.b64decode(data.split(',')[1]))
    with open(filename, 'wb') as f:
        f.write(binary.getbuffer())
    return filename

# Cell 4: Pose and fall detection logic
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

def detect_fall(landmarks):
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    if left_hip.visibility < 0.5 or right_hip.visibility < 0.5:
        return False
    avg_hip_y = (left_hip.y + right_hip.y) / 2
    return avg_hip_y > 0.75

def train_model():
    print("🧠 Training model (simulated)...")
    for i in range(3):
        print(f"Epoch {i+1}/3")
        time.sleep(1)
    print("✅ Model training complete.")

def webcam_test_colab():
    print("📸 Please allow camera and click 'Take Photo'")
    photo_path = take_photo()
    print('✅ Photo saved to:', photo_path)

    image = cv2.imread(photo_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)

    fall_detected = False
    if result.pose_landmarks:
        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        fall_detected = detect_fall(result.pose_landmarks.landmark)

        if fall_detected:
            cv2.putText(image, 'FALL DETECTED!', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
        else:
            cv2.putText(image, 'NO FALL DETECTED', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
    else:
        cv2.putText(image, 'NO PERSON DETECTED', (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 4)

    cv2_imshow(image)

    if fall_detected:
        print("🚨 FALL DETECTED!")
    else:
        print("✅ No fall detected.")

def test_on_video(video_path):
    cap = cv2.VideoCapture(video_path)
    pose_video = mp.solutions.pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
    fall_detected = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose_video.process(frame_rgb)
        if results.pose_landmarks:
            if detect_fall(results.pose_landmarks.landmark):
                fall_detected = True
                print("🚨 Fall detected in video!")
        cv2.imshow('Video Fall Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    if not fall_detected:
        print("✅ No fall detected in video.")

# Cell 5: Menu
def main():
    print(" Fall Detection Menu:")
    print("1️⃣ Train")
    print("2️⃣ Test with Webcam")
    print("3️⃣ Test on Video File")
    choice = input("Enter your choice (1/2/3): ").strip()

    if choice == "1":
        train_model()
    elif choice == "2":
        webcam_test_colab()
    elif choice == "3":
        video_path = input("Enter path to video file (.mp4): ").strip()
        test_on_video(video_path)
    else:
        print("❌ Invalid choice. Please enter 1, 2 or 3.")

# Run the menu
main()

 Fall Detection Menu:
1️⃣ Train
2️⃣ Test with Webcam
3️⃣ Test on Video File
Enter your choice (1/2/3): 2
📸 Please allow camera and click 'Take Photo'


<IPython.core.display.Javascript object>

AttributeError: module 'io' has no attribute 'Bytes'

In [7]:
!pip install mediapipe opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: numpy
    Found exis

In [ ]:
!pip install mediapipe opencv-python-headless

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_